# Efficient Attention Mechanism

> An efficient implementation of attention mechanism

In [2]:
#| default_exp transformer.efficient_attention

In [3]:
#| hide
from nbdev.showdoc import *

In [5]:
#| hide
import nbdev; nbdev.nbdev_export()

In [7]:
#| export
import math

import torch
from torch import nn
import torch.nn.functional as F

In [10]:
#| export
class ScaleDotProductAttention(nn.Module):
    def __init__(self, d_head: int):
        super().__init__()
        self.d_head = d_head
    
    def forward(self, q_batch: torch.Tensor, k_batch: torch.Tensor, v_batch: torch.Tensor, mask=None):
        # shape(q_batch) = [batch_size x num_heads x Q_len x d_head]
        # shape(k_batch, v_batch) = [batch_size x num_heads x KV_len x d_head]
        
        # shape(K_permuted) = [batch_size x num_heads x d_head x length]
        k_batch_permuted = k_batch.permute(0, 1, 3, 2)
        q_k_matmul_batch = torch.matmul(q_batch, k_batch_permuted)
        scores_batch = q_k_matmul_batch / math.sqrt(self.d_head)
        
        if mask is not None:
            scores_batch = scores_batch.masked_fill(mask == 0, -1e9)
        
        attention_weights_batch = F.softmax(scores_batch, dim=-1)
        output_batch = torch.matmul(attention_weights_batch, v_batch)
        
        return output_batch, attention_weights_batch

In [11]:
#| export
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model: int, n_head: int, dropout: float = 0.3):
        super().__init__()
        self.n_head = n_head
        self.d_model = d_model
        self.d_head = d_model // n_head
        
        self.attention = ScaleDotProductAttention(d_head = self.d_head)
        
        self.dropout = nn.Dropout(dropout)
        
        self.w_q = nn.Linear(d_model, d_model)
        self.w_k = nn.Linear(d_model, d_model)
        self.w_v = nn.Linear(d_model, d_model)
        self.w_concat = nn.Linear(d_model, d_model)
        
        self.linear_projection = nn.Linear(d_model, d_model)
    
    def split_head(self, x: torch.Tensor):
        batch_size, n_words, d_model = x.size()
        d_head = d_model // self.n_head
        
        tenxsor = x.view(batch_size, n_words, self.n_head, d_head)
        x = x.transpose(2, 1) # transpose self.n_head and n_words
        
        return x

    def concat(self, x: torch.Tensor):
        batch_size, n_heads, n_words, d_head = x.size()
        return x.view(batch_size, -1, self.d_model)
    
    def forward(self, pre_q: torch.Tensor, pre_k: torch.Tensor, pre_v: torch.Tensor, mask=None):
        
        # 1. dot product with weight matrices
        q_batch, k_batch, v_batch = self.w_q(pre_q), self.w_k(pre_k), self.w_v(pre_v)
        
        # 2. split tensor by number of heads
        q_batch, k_batch, v_batch = self.split_head(q_batch), self.split_head(k_batch), self.split_head(v_batch)
        
        output_batch, attention_weights_batch = self.attention(q_batch, k_batch, v_batch, mask)
        output_batch = self.concat(output_batch)
        
        projection_batch = self.dropout(self.linear_projection(output_batch))
        
        return projection_batch, attention_weights_batch